In [1]:
from SQLite_handler import *
import rapidjson as json
from tqdm import tqdm
import os

# 0 > JB; 1 > DT
US_states = ['NY', 'DC', 'IN', 'AR', 'WY', 'ME', 'TX', 'NH', 'CO', 'CA', 'IL',
             'WA', 'VA', 'FL', 'MA', 'OR', 'AZ', 'MT', 'MN', 'NE', 'TN', 'OH',
             'NJ', 'NV', 'KY', 'UT', 'NC', 'SC', 'PA', 'NM', 'KS', 'GA', 'MI',
             'WI', 'AK', 'MS', 'MD', 'LA', 'HI', 'MO', 'AL', 'CT', 'OK', 'IA',
             'WV', 'RI', 'SD', 'VT', 'ND', 'ID', 'DE']

In [7]:
user_info = {}
for line in tqdm(open("D:/US2020/user-state.txt", encoding='utf8')):
    w = line.strip().split("\t")
    user_info[w[0]] = w[-1] # 州结果

6739702it [00:10, 652291.94it/s]


In [8]:
json_file_name = 'data/users-14days-v1-0.5/2020-10-26.json'

user_result = json.load(open(json_file_name, encoding='utf8'))
u_table = []
for u, v in user_result.items():
    if u not in user_info:
        continue
    u_state = user_info[u]
    if v[0] > v[1]:
        opinion = "B"
    elif v[0] < v[1]:
        opinion = "T"
    elif v[0] == v[1]:
        opinion = "U"

    u_table.append({
        "uid": u,
        "state": u_state,
        "opinion": opinion
    })

In [11]:
u_df = pd.DataFrame(u_table).set_index('uid')
u_result = u_df.groupby("state")["opinion"].value_counts()

names = json_file_name.split("/")
u_result.to_csv(f"data/state_results/{names[-2]}-{names[-1][:-5]}.csv")

In [5]:
u_df = pd.DataFrame(u_table).set_index('uid')

In [6]:
u_df

,state,opinion
uid,,
562430983,"[562430983, quarantined, Quarantine Hill, Wick...",T
401728487,"[401728487, st. petersburg, fl, St Petersburg,...",B
824331320771878914,"[824331320771878914, alabama, usa, AL, United ...",B
62737072,"[62737072, oakland , Oakland, CA, United State...",B
115531443,"[115531443, united states, United States, None]",T
...,...,...
322979719,"[322979719, university of houston, Houston, TX...",B
874348789871652865,"[874348789871652865, london, england, London, ...",T
451658476,"[451658476, delaware, usa, DE, United States, ...",B
